In [172]:
import os
from collections import OrderedDict

import numpy as np
import pandas as pd
import requests
import seaborn as sns
import xarray as xr

% matplotlib inline

### WB Data Analysis

**<span style="color:red; background:yellow;"></span>**

**Data & Scope:** downloaded on (date/time)
Two types of datasets:

1) Population (easy)
* years 1960-2016
* definition: total # residents (Regardless of citizenship/legal status)
* sources: (mid-year value)

    1) United Nations Population Division. World Population Prospects
    
    2) Census reports and other statistical publications from national statistical offices
    
    3) Eurostat: Demographic Statistics
    
    4) United Nations Statistical Division. Population and Vita

2) GDP (slightly more tricky)
- years: 1960 - 2016
- four types of `gdp per capita` I looked at:

    * current
    * constant
    * PPP current
    * PPP constant

**Source:** World Bank

**Assumptions/Expectations**
-

**Analysis Goals**
- [x] Task 3 Deliverable: ADM0 population & real income estimates from `1950-2017` 
- [] Find out which countries are missing data for which years.
- [] Or alternatively, which years are missing certain *countries*

Possible convenience functions:
* Find years missing for a given country (to check important countries)
* Create filter for population > 10 mil: look at countries exit/enter data on which years. (Mike's suggesiton)

**Conclusion**
1. Population. Population data exists from **1960 to 2016.**
    
```  
Countries missing population data (nan):

West Bank and Gaza: 1960 - 1989
Serbia: 1960 - 1989
Sint Maarten (Dutch part): 1960 -1998
Kuwait: 1992-1994
Eritrea: 2012-2016

```

2. GDP


**Questions**
1. How are different sources of population data used to result in one final set? (are there any overlaps between 4 sources?) i.e. what is the methodology for compilation?

2. How often are population/income data updated?



# Population data

Total population (absolute units) - based on national census and extrapolation and interpolation for missing values (based on data from United Nations, other census organizations, Eurostats and WB methodology). Subject to undercounting/biases for both high and low/mid income countries. 

Interpolation and extrapolation done by World Bank/UN (??-confirm the responsible party) for certain years/countries that are missing census data, or missing pre/post census information for given time frame. Uses demographic models, etc. 

In [173]:
df2 = pd.read_csv('population/API_SP.POP.TOTL_DS2_en_csv_v2.csv', skiprows=3)
del df2['Unnamed: 62'] # remove extraneous data
df2.columns


Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017'],
      dtype='object')

In [174]:
df2
df2['2017'].unique() # nan -> all 2017 values nan

array([ nan])

In [175]:
cols = ['Country Name'] + [str(yr) for yr in range(1960, 2017)]
#print (cols)
df_pop = df2[cols]
df_pop # includes Country Name and all years (1960-2016)
df_pop.shape # 264 countries, 58 columns 

(264, 58)

In [176]:
df_pop.head(5)   # Show top 5 rows
df_pop.isnull().sum()
# note that nan values per year changes from 1-4, 1989 and prior are all 4,
# after that there is a shifts six times:
# 4 (60-89) -> 2 (90-91)-> 3 (92-94) -> 2 (95-97) -> 1 (98-2011)-> 2 (2012-2016)-> 264 (2017)

# Compare two years (diff countries)

df_pop['1989'].isnull()
pd.isnull(df_pop).any(1).nonzero()[0]

array([ 67, 108, 125, 194, 212, 223])

In [177]:
df_pop.iloc[67] # Eritrea missing 2012-2016
df_pop.iloc[108] # Not classified missing all # what is this....?
df_pop.iloc[194] # West Bank and Gaza missing 1960 - 1989
df_pop.iloc[212] # Serbia missing 1960 - 1989
df_pop.iloc[223] # Sint Maarten (Dutch part) 1960 -1998
df_pop.iloc[125] # Kuwait missing 1992-1994

# doing this manual painful way for now
df_pop['1960'].isnull().sum() # 4 
df_pop['1990'].isnull().sum() # expect 2 (excluding West Bank/Serbia)
df_pop['1993'].isnull().sum() # expect 3 (Kuwait missing too)
df_pop['1995'].isnull().sum() # expect 2 (Kuwait back again)
df_pop['1999'].isnull().sum() # expect 1 Sint Maarten back
df_pop['2012'].isnull().sum() # expect 2 (Eritrea goes missing)
df_pop['2016'].isnull().sum() # expect 2

# drop unclassified country since it's contributing 0 population 
df_pop.drop(df_pop.index[108])

#print(df_pop['Country Name'].tolist())

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,5.421100e+04,5.543800e+04,5.622500e+04,5.669500e+04,5.703200e+04,5.736000e+04,5.771500e+04,5.805500e+04,5.838600e+04,...,1.012200e+05,1.013530e+05,1.014530e+05,1.016690e+05,1.020530e+05,1.025770e+05,1.031870e+05,1.037950e+05,1.043410e+05,1.048220e+05
1,Afghanistan,8.996351e+06,9.166764e+06,9.345868e+06,9.533954e+06,9.731361e+06,9.938414e+06,1.015233e+07,1.037263e+07,1.060435e+07,...,2.661679e+07,2.729403e+07,2.800433e+07,2.880317e+07,2.970860e+07,3.069696e+07,3.173169e+07,3.275802e+07,3.373649e+07,3.465603e+07
2,Angola,5.643182e+06,5.753024e+06,5.866061e+06,5.980417e+06,6.093321e+06,6.203299e+06,6.309770e+06,6.414995e+06,6.523791e+06,...,2.099769e+07,2.175942e+07,2.254955e+07,2.336913e+07,2.421856e+07,2.509615e+07,2.599834e+07,2.692047e+07,2.785930e+07,2.881346e+07
3,Albania,1.608800e+06,1.659800e+06,1.711319e+06,1.762621e+06,1.814135e+06,1.864791e+06,1.914573e+06,1.965598e+06,2.022272e+06,...,2.970017e+06,2.947314e+06,2.927519e+06,2.913021e+06,2.905195e+06,2.900401e+06,2.895092e+06,2.889104e+06,2.880703e+06,2.876101e+06
4,Andorra,1.341100e+04,1.437500e+04,1.537000e+04,1.641200e+04,1.746900e+04,1.854900e+04,1.964700e+04,2.075800e+04,2.189000e+04,...,8.268300e+04,8.386100e+04,8.446200e+04,8.444900e+04,8.375100e+04,8.243100e+04,8.078800e+04,7.922300e+04,7.801400e+04,7.728100e+04
5,Arab World,9.249093e+07,9.504450e+07,9.768229e+07,1.004111e+08,1.032399e+08,1.061750e+08,1.092306e+08,1.124069e+08,1.156802e+08,...,3.316538e+08,3.398255e+08,3.481451e+08,3.565089e+08,3.648959e+08,3.733070e+08,3.817021e+08,3.900430e+08,3.983050e+08,4.064527e+08
6,United Arab Emirates,9.263400e+04,1.010780e+05,1.124720e+05,1.255660e+05,1.385290e+05,1.503620e+05,1.604810e+05,1.702830e+05,1.831940e+05,...,6.044067e+06,6.894278e+06,7.666393e+06,8.270684e+06,8.672475e+06,8.900453e+06,9.006263e+06,9.070867e+06,9.154302e+06,9.269612e+06
7,Argentina,2.061908e+07,2.095308e+07,2.128768e+07,2.162184e+07,2.195393e+07,2.228339e+07,2.260875e+07,2.293220e+07,2.326128e+07,...,3.997022e+07,4.038239e+07,4.079941e+07,4.122389e+07,4.165688e+07,4.209674e+07,4.253992e+07,4.298152e+07,4.341776e+07,4.384743e+07
8,Armenia,1.874120e+06,1.941491e+06,2.009526e+06,2.077575e+06,2.144998e+06,2.211316e+06,2.276031e+06,2.339124e+06,2.401140e+06,...,2.933056e+06,2.908220e+06,2.888584e+06,2.877311e+06,2.875581e+06,2.881922e+06,2.893509e+06,2.906220e+06,2.916950e+06,2.924816e+06
9,American Samoa,2.001300e+04,2.048600e+04,2.111700e+04,2.188200e+04,2.269800e+04,2.352000e+04,2.432100e+04,2.511600e+04,2.588500e+04,...,5.790300e+04,5.703000e+04,5.622700e+04,5.563700e+04,5.532000e+04,5.523000e+04,5.530700e+04,5.543700e+04,5.553700e+04,5.559900e+04


### Getting data for country

In [178]:
# Returns all values of the 'Country Name' column as an array
#df2['Country Name'].values #lists all countries
# United States
# China
# India

china = get_row_by_country_name(df2, 'China')
india = get_row_by_country_name(df2, 'India')
usa = get_row_by_country_name(df2, 'United States')

def get_first_and_last_years(_country):
    # _country is a dataframe (1 row)
    return _country[['1960', '2016']]
    
    
get_first_and_last_years(china)
get_first_and_last_years(usa)
get_first_and_last_years(india)

# manual confirmation: these values match online WB values (https://data.worldbank.org/indicator/SP.POP.TOTL?year_high_desc=true)

,1960,2016
107,449480608.0,1.324171e+09


# GDP data

### GDP per capita PPP (constant at 2011 `$`)

In [179]:
# Open file
df = pd.read_csv('./gdp/gdp_pcap_ppp_constant/API_NY.GDP.PCAP.PP.KD_DS2_en_csv_v2.csv', skiprows=3)

In [180]:
# Get variables (column names)
len(df.columns) # 63 variables

#df.columns # years 1960-2017

len(df['Country Name'])

264

### Get list of years missing for a country



In [181]:
# Series.value_counts() -> get unique counts of values

# Count nan
sorted(df.isnull().sum(axis=1).unique().tolist()) # 17 unique values
# Why do some countries have randomly missing years? (source of inconsistency?)

[32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 44, 46, 49, 51, 58, 59]

In [182]:
# Get all rows 
# Get selected columns (ranging from 1960 to 2017)
df_1960_2017 = df.loc[:, '1960':'2017']

#df_1960_2017.apply(lambda s: s.value_counts(), axis=0)

df_1960_2017.shape # 264, 58 -> 264 countries and 58 years
num_years = 2017 - 1960 + 1 # 58
#print('num years is {}'.format(num_years))
#df_1960_2017.shape[1] # 58 (counts columns) # 1-> colum

df_1960_2017.apply(lambda s: s.value_counts(), axis=0)
#print(len(df_1960_2017.isnull().sum(axis=1).tolist())) # 264 = # countries

# returns list of # nan per country
num_nan_per_country = df_1960_2017.isnull().sum(axis=1).tolist()
print(df_1960_2017.isnull().sum(axis=1).tolist())

list_of_countries = [x for x in df['Country Name'].values]
num_countries = len(list_of_countries)

countries = {list_of_countries[i]: {'num_missing': num_nan_per_country[i]} for i in range(0, num_countries)}

countries

[57, 43, 31, 31, 58, 31, 31, 31, 31, 58, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 31, 35, 31, 31, 34, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 58, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 58, 58, 57, 31, 31, 31, 32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 38, 31, 36, 31, 31, 31, 31, 31, 31, 58, 31, 31, 31, 31, 31, 58, 31, 31, 31, 31, 31, 31, 58, 31, 58, 31, 31, 31, 31, 31, 36, 37, 32, 31, 31, 31, 31, 31, 31, 58, 31, 58, 31, 32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 34, 31, 31, 31, 37, 31, 31, 31, 31, 45, 31, 31, 31, 31, 58, 31, 31, 31, 31, 31, 36, 31, 36, 31, 58, 31, 58, 36, 31, 42, 31, 31, 31, 31, 31, 31, 31, 31, 32, 38, 31, 58, 31, 31, 31, 31, 31, 31, 31, 58, 31, 31, 31, 31, 31, 31, 48, 31, 31, 32, 41, 31, 31, 31, 31, 32, 33, 31, 31, 34, 58, 31, 31, 58, 31, 31, 58, 41, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 58, 58, 36, 31, 50, 31, 41, 41, 31, 33, 36, 31, 31, 57, 31, 58, 58, 31, 31, 31, 31, 31, 31, 31, 31, 41, 32, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,

{'Afghanistan': {'num_missing': 43},
 'Albania': {'num_missing': 31},
 'Algeria': {'num_missing': 31},
 'American Samoa': {'num_missing': 58},
 'Andorra': {'num_missing': 58},
 'Angola': {'num_missing': 31},
 'Antigua and Barbuda': {'num_missing': 31},
 'Arab World': {'num_missing': 31},
 'Argentina': {'num_missing': 31},
 'Armenia': {'num_missing': 31},
 'Aruba': {'num_missing': 57},
 'Australia': {'num_missing': 31},
 'Austria': {'num_missing': 31},
 'Azerbaijan': {'num_missing': 31},
 'Bahamas, The': {'num_missing': 31},
 'Bahrain': {'num_missing': 32},
 'Bangladesh': {'num_missing': 31},
 'Barbados': {'num_missing': 31},
 'Belarus': {'num_missing': 31},
 'Belgium': {'num_missing': 31},
 'Belize': {'num_missing': 31},
 'Benin': {'num_missing': 31},
 'Bermuda': {'num_missing': 34},
 'Bhutan': {'num_missing': 31},
 'Bolivia': {'num_missing': 31},
 'Bosnia and Herzegovina': {'num_missing': 35},
 'Botswana': {'num_missing': 31},
 'Brazil': {'num_missing': 31},
 'British Virgin Islands':

In [183]:
def get_missing_years(country_name):
    '''
    Given a start_year and an end_year, calculate the number of years in 
    the given range (inclusive at both ends) the country has no
    data points (ie. has nan value)
    
    Parameters
    ----------
    country_name : Str
        name of given country
        
    Returns 
    -------
    Missing years in default range (1960-2017), where 
    each year is a string : List
    
    ex. ['1990',  '2005'']

'''
    country = df.loc[df['Country Name'] == country_name]
    return country

# Get countries as dataframe (subset of original) by first letter
def get_by_first_letter(df, first_letter):
    return df.loc[df['Country Name'].str.startswith(first_letter)]

# Get country by name
def get_row_by_country_name(_df, country_name):
    return _df[_df['Country Name'] == country_name]

# Get country row from dataframe by index
def get_by_index(df, country_idx):
    return df.iloc[country_idx]

# Get all countries that start with S
s_countries = get_by_first_letter(df, 'S')
#s_countries
    
# Get_by_first_letter(df, 'K') # example South Korea is index 124
get_by_index(df, 124)


Country Name                                            Korea, Rep.
Country Code                                                    KOR
Indicator Name    GDP per capita, PPP (constant 2011 internation...
Indicator Code                                    NY.GDP.PCAP.PP.KD
1960                                                            NaN
1961                                                            NaN
1962                                                            NaN
1963                                                            NaN
1964                                                            NaN
1965                                                            NaN
1966                                                            NaN
1967                                                            NaN
1968                                                            NaN
1969                                                            NaN
1970                                            

In [184]:
# Write a function that counts values for each column
# Purpose: to count how many NA values are there in each row
df['1960'].value_counts()
type(df.iloc[0]['1960']) # numpy.float64

# note: comparison of nan == np.nan does not work (returns False)
# need to check np.isnan(val)

# single value check
np.isnan(df.iloc[0]['1960']) # True 

# returns total number of nan values for each column (whole df)
df.isnull().sum() 

# find the countries that have nan values in 1989 and prior years, 
# but don't have 


Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              264
1961              264
1962              264
1963              264
1964              264
1965              264
1966              264
1967              264
1968              264
1969              264
1970              264
1971              264
1972              264
1973              264
1974              264
1975              264
1976              264
1977              264
1978              264
1979              264
1980              264
1981              264
1982              264
1983              264
1984              264
1985              264
                 ... 
1989              264
1990               55
1991               53
1992               51
1993               50
1994               49
1995               41
1996               40
1997               39
1998               39
1999               37
2000               32
2001               31
2002               30
2003      

In [185]:
189 + 28 # 217 number of countries

217

## B. Population DATA

### Getting data for country

In [186]:
# Returns all values of the 'Country Name' column as an array
#df2['Country Name'].values #lists all countries
# United States
# China
# India

china = get_row_by_country_name(df2, 'China')
india = get_row_by_country_name(df2, 'India')
usa = get_row_by_country_name(df2, 'United States')

def get_first_and_last_years(_country):
    # _country is a dataframe (1 row)
    return _country[['1960', '2016']]
    
    
get_first_and_last_years(china)
get_first_and_last_years(usa)
get_first_and_last_years(india)

# manual confirmation: these values match online WB values (https://data.worldbank.org/indicator/SP.POP.TOTL?year_high_desc=true)

,1960,2016
107,449480608.0,1.324171e+09


### Creating timeseries visualization

In [187]:
sns.set(color_codes=True)
x = np.linspace(1960, 2016, 57) # returns array of values from 1960 to 2016 (inclusive)
# 57 determines the total number of values we want (i.e. (2016-1960+1)/57 = 1)
# same as np.arange(1960, 2017, 1) # 1960 to 2017 (not including 2017), specifies step=1
# same as np.arange(1960, 2017) # default step is 1

# For fun, plot China, US, and India population change
#ax = sns.tsplot(data=china)


### Fetching data via API

This approach is abandoned because json data is harder to manipulate for this analysis.
Pandas rocks.

In [188]:
#r = requests.get('http://api.worldbank.org/v2/countries/all/indicators/SP.POP.TOTL?format=json', auth=('user', 'pass'))

In [189]:
#r.headers['Content-Type'] # 'application/json;charset=utf-8'
#r.json()